In [84]:
import pandas as pd
import numpy as np

import inflection

In [4]:
df = pd.read_csv('../dataset/zomato.csv')

In [85]:
df2 = pd.read_csv('../dataset/data.csv')

In [5]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [6]:
df1 = df.copy()

In [7]:
COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
    
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

df1 = rename_columns(df1);
df1['country'] = df1['country_code'].map(COUNTRIES);

# removing outlier
df1.loc[385, ['average_cost_for_two']] = 250

In [46]:
df1['cuisines'] = df1['cuisines'].apply(lambda x: str(x).split(', ')[0])

In [8]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country'],
      dtype='object')

In [9]:
# 1. Quantos restaurantes únicos estão registrados?
unique_restaurant = df1['restaurant_id'].nunique()

# 2. Quantos países únicos estão registrados?
countries = df1['country_code'].nunique()

# 3. Quantas cidades únicas estão registradas?
cities = df1['city'].nunique()

# 4. Qual o total de avaliações feitas?
votes = df1['votes'].sum()

# 5. Qual o total de tipos de culinária registrados?
cuisines = df1['cuisines'].nunique()


In [82]:
# 1. Qual o nome do país que possui mais cidades registradas?
cities_by_country = df1[['city', 'country']].groupby('country').count().sort_values('city', ascending=False).reset_index()

# 2. Qual o nome do país que possui mais restaurantes registrados?
restaurants_by_country = df1[['restaurant_id', 'country']].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index()

# 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?
exp_rest_by_country = df1[['restaurant_id', 'country']].loc[df1['price_range'] == 4].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index()

# 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?
unique_cuisines = df1[['country', 'cuisines']].groupby('country').nunique().sort_values('cuisines', ascending=False).reset_index()

# 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?
unique_avs = df1[['country', 'votes']].groupby('country').sum().sort_values('votes', ascending=False).reset_index()

# 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?
unique_rests = df1[['country', 'restaurant_id']].groupby('country').nunique().reset_index()

# 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?
reservation_rests = df1[['country', 'restaurant_id',]].loc[df1['has_table_booking'] == 1].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index()

# 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

# 9. Qual o nome do país que possui, na média, a maior nota média registrada?

# 10. Qual o nome do país que possui, na média, a menor nota média registrada?

# 11. Qual a média de preço de um prato para dois por país?
mean_price_for_two = df1[['country', 'average_cost_for_two']].loc[df1['average_cost_for_two'] != 0].groupby('country').mean().sort_values('average_cost_for_two').reset_index()

In [10]:
def process_data(file_path):
    df = pd.read_csv(file_path)

    df = df.dropna()

    df = rename_columns(df)

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x))

    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    df = df.drop_duplicates()

    df.to_csv("../dataset/data.csv", index=False)

    return df

In [37]:
aux_map = df1[['latitude', 'longitude', 'aggregate_rating']].copy()
aux_map.rename( columns = {'latitude': 'lat', 'longitude': 'lon'}, inplace = True )
aux_map['aggregate_rating'] = aux_map['aggregate_rating'] 
aux_map.head()

,lat,lon,aggregate_rating
0,14.447615,121.009787,4.6
1,14.447615,121.009787,4.6
2,14.556042,121.024562,4.7
3,14.556196,121.023171,4.4
4,14.565899,121.027708,4.4


In [83]:
# df1[['country', 'price_range', 'average_cost_for_two']].loc[df1['country'] == 'Australia'].sort_values('average_cost_for_two', ascending=False)
mean_price_for_two

,country,average_cost_for_two
0,Canada,42.570621
1,England,43.950935
2,United States of America,55.318412
3,New Zeland,62.681452
4,Australia,73.412429
5,Turkey,128.848485
6,Singapure,141.036585
7,Brazil,141.161417
8,United Arab Emirates,161.112805
9,Qatar,176.648936


In [87]:
df1['price_range']

0       3
1       3
2       4
3       3
4       3
       ..
7522    4
7523    2
7524    4
7525    3
7526    4
Name: price_range, Length: 7527, dtype: int64